## We need the lat-lon wind for plotting streamlines and calculating Liapunov components

In [19]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs

import cubedsphere as cs
import xesmf as xe

In [20]:
maindir = '/n/home03/zhuangjw/seasasfs/fromPleiades/output_FV3/' # Odyssey

#datadir = maindir+'C48/C48L20/'
datadir = maindir+'/C384/C384L20/'
#datadir = maindir+'/C384/spinup/'

ds = cs.open_FV3data(datadir,'winds_daily')
ds

<xarray.Dataset>
Dimensions:  (pfull: 20, phalf: 21, tile: 6, time: 8, x: 384, x_b: 385, y: 384, y_b: 385)
Coordinates:
  * x        (x) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * y        (y) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 ...
  * pfull    (pfull) float64 7.673 47.07 100.7 152.7 204.4 255.8 307.2 358.6 ...
  * phalf    (phalf) float64 1.0 26.0 77.26 128.5 179.8 231.1 282.3 333.6 ...
  * time     (time) float64 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0
    lon      (tile, y, x) float32 305.097 305.293 305.488 305.685 305.881 ...
    lat      (tile, y, x) float32 -35.2184 -35.3098 -35.4009 -35.4917 ...
  * x_b      (x_b) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * y_b      (y_b) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
    lon_b    (tile, y_b, x_b) float32 305.0 305.195 305.391 305.586 305.783 ...
    lat_b    (tile, y_b, x_b) float32 -35.2644 -35.3561 -35.4474 -35.5385 ...
    area     (tile, y, x) floa

In [21]:
#ds_out = xe.grid_global(2, 2)
ds_out = xe.grid_global(0.25, 0.25)
ds_out

<xarray.Dataset>
Dimensions:  (x: 1440, x_b: 1441, y: 720, y_b: 721)
Coordinates:
    lon      (y, x) float64 -179.9 -179.6 -179.4 -179.1 -178.9 -178.6 -178.4 ...
    lat      (y, x) float64 -89.88 -89.88 -89.88 -89.88 -89.88 -89.88 -89.88 ...
    lon_b    (y_b, x_b) float64 -180.0 -179.8 -179.5 -179.2 -179.0 -178.8 ...
    lat_b    (y_b, x_b) float64 -90.0 -90.0 -90.0 -90.0 -90.0 -90.0 -90.0 ...
Dimensions without coordinates: x, x_b, y, y_b
Data variables:
    *empty*

In [22]:
def C2L(ds_in, ds_out, dr_in):
    
    for k in range(6):
        print(k, end=' ')
        dr_temp = xe.regrid(ds_in.isel(tile=k), ds_out, 
                            dr_in.isel(tile=k), method='conservative')
        if k==0:
            dr_out = dr_temp
        else:
            dr_out += dr_temp
    
    return  dr_out

In [23]:
# load into memory before regridding
dr_u = ds['u700'].isel(time=-1).load()
dr_v = ds['v700'].isel(time=-1).load()

In [27]:
# take very long for high resolution (~10min for C384->0.25x0.25)
u_out = C2L(ds, ds_out, dr_u)
v_out = C2L(ds, ds_out, dr_v)

0 1 2 3 4 5 0 1 2 3 4 5 

In [28]:
# record into dataset
ds_out['u700'] = u_out
ds_out['v700'] = v_out

In [29]:
ds_out

<xarray.Dataset>
Dimensions:  (x: 1440, x_b: 1441, y: 720, y_b: 721)
Coordinates:
    lon      (y, x) float64 -179.9 -179.6 -179.4 -179.1 -178.9 -178.6 -178.4 ...
    lat      (y, x) float64 -89.88 -89.88 -89.88 -89.88 -89.88 -89.88 -89.88 ...
    lon_b    (y_b, x_b) float64 -180.0 -179.8 -179.5 -179.2 -179.0 -178.8 ...
    lat_b    (y_b, x_b) float64 -90.0 -90.0 -90.0 -90.0 -90.0 -90.0 -90.0 ...
Dimensions without coordinates: x, x_b, y, y_b
Data variables:
    u700     (y, x) float64 0.739 0.739 0.739 0.739 0.739 0.739 0.739 0.739 ...
    v700     (y, x) float64 -0.2885 -0.2885 -0.2885 -0.2885 -0.2885 -0.2885 ...

In [30]:
outputdir = '../../outputdata/regridded/'
#ds_out.to_netcdf(outputdir+'C48L20_wind700.nc')
ds_out.to_netcdf(outputdir+'C384L20_wind700.nc')